In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix , ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression



In [ ]:
churn=pd.read_csv('Churn_Modelling.csv')
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


# **DATA CLEANING**

In [ ]:
#הסרתי את שלושת העמודת כי הם לא רלוונטיות לחישוב שלי והם רק שם ומספר שורה
churn.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
churn.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#יש יתרון בהשלמה ולא הסרה כך שאנחנו לא מוותרים על נתונים ולפעמים זה יכול להיות משמעותי
#יש חסרון בהשלמה שלפעמים הנתונים לא באמת מדויקים ומייציגים באמת את המציאות
#3 גישות שונות להשלמת נתונים :
#חציון:נשתמש כאשר מדובר במשתנה מספרי וזה טוב כי זה הרוב בדכ
#שכיח :נשתתמש כאשר מדובר במשתנה קטגוריאלי וזה טוב כי יש הכי הרבה ממנו והסיכוי שנפגע הוא גבוהה מאוד
#השלמה לפני נתונים קודמים:נשתמש כאשר המשתנים הם לפי רצף זמן וזה טוב כי לרוב המון משתנים חוזרים על עצמם במשך זמן שמודדים אותם
churn.isnull().sum()

,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0
HasCrCard,0
IsActiveMember,0
EstimatedSalary,0


In [ ]:
churn['Geography'].value_counts()


,count
Geography,
France,5014
Germany,2509
Spain,2477


In [ ]:
churn['Gender'].value_counts()

,count
Gender,
Male,5457
Female,4543


In [ ]:
churn=pd.get_dummies(churn,columns=['Geography'],drop_first=True,dtype=int)
churn.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0,1


In [ ]:
churn['Gender']=churn['Gender'].map({'Female':0,'Male':1})
churn.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


In [ ]:
churn.describe()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,0.545700,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,0.250900,0.247700
std,96.653299,0.497932,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.433553,0.431698
min,350.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000
25%,584.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000
50%,652.000000,1.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,0.000000,0.000000
75%,718.000000,1.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.000000,0.000000
max,850.000000,1.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,1.000000


In [ ]:
# prompt: put the exited coulmn as the last coulmn

# Assuming 'Exited' is the column you want to move to the last position

# Get a list of column names
cols = list(churn.columns)

# Remove 'Exited' from its current position
cols.remove('Exited')

# Append 'Exited' to the end of the list
cols.append('Exited')

# Reorder the DataFrame using the new column order
churn = churn[cols]

churn.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,0,0,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0,1
3,699,0,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,1,0


In [ ]:
X=churn.iloc[:,1:-1]
X.head()

,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,0,42,2,0.00,1,1,1,101348.88,0,0
1,0,41,1,83807.86,1,0,1,112542.58,0,1
2,0,42,8,159660.80,3,1,0,113931.57,0,0
3,0,39,1,0.00,2,0,0,93826.63,0,0
4,0,43,2,125510.82,1,1,1,79084.10,0,1


In [ ]:
y=churn.iloc[:,-1]
y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [ ]:
#רק משתנה איקס זקוק לנירמול מיכווין שיש המון משתנים מספריים שלא בהרכח יש להם את אותו משקל אחד כלפי השני
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
X_scaled=pd.DataFrame(X_scaled,columns=X.columns)
X_scaled.head()

,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,0.0,0.324324,0.2,0.000000,0.000000,1.0,1.0,0.506735,0.0,0.0
1,0.0,0.310811,0.1,0.334031,0.000000,0.0,1.0,0.562709,0.0,1.0
2,0.0,0.324324,0.8,0.636357,0.666667,1.0,0.0,0.569654,0.0,0.0
3,0.0,0.283784,0.1,0.000000,0.333333,0.0,0.0,0.469120,0.0,0.0
4,0.0,0.337838,0.2,0.500246,0.000000,1.0,1.0,0.395400,0.0,1.0


In [ ]:
#פעולת הנירמול חשובה באלגוריתמים הבאים:
#KNN:מחשבת מרחק ולכן חשוב שכל הנתונים יקבלו את אותו המשקל שצריך על מנת שלא יהיו סטיות גדולות מידי
#K-MEANS:עובד על מרחק ולכן יש צורך גם לנרמל את נתונים

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_scaled,
                                                    y,
                                                    test_size=0.2
                                                ,
                                                    random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 10)
(2000, 10)
(8000,)
(2000,)


In [ ]:
Ks=list(range(1,52,2))
len(Ks)

26

In [ ]:
Accuracies=[]
for K in Ks:
    KNN=KNeighborsClassifier(n_neighbors=K)
    KNN.fit(x_train,y_train)
    Score=KNN.score(x_test, y_test)
    Accuracies.append(Score)
    print("K=",K)
    print("Accuracy:", round(Score,2))
best_accurcy=max(Accuracies)
best_k=Ks[Accuracies.index(best_accurcy)]
print("Best K:",best_k)
print("Best Accuracy:",round(best_accurcy,2))

K= 1
Accuracy: 0.8
K= 3
Accuracy: 0.82
K= 5
Accuracy: 0.83
K= 7
Accuracy: 0.82
K= 9
Accuracy: 0.83
K= 11
Accuracy: 0.83
K= 13
Accuracy: 0.83
K= 15
Accuracy: 0.83
K= 17
Accuracy: 0.82
K= 19
Accuracy: 0.83
K= 21
Accuracy: 0.82
K= 23
Accuracy: 0.83
K= 25
Accuracy: 0.83
K= 27
Accuracy: 0.83
K= 29
Accuracy: 0.82
K= 31
Accuracy: 0.82
K= 33
Accuracy: 0.82
K= 35
Accuracy: 0.82
K= 37
Accuracy: 0.82
K= 39
Accuracy: 0.82
K= 41
Accuracy: 0.82
K= 43
Accuracy: 0.82
K= 45
Accuracy: 0.82
K= 47
Accuracy: 0.82
K= 49
Accuracy: 0.82
K= 51
Accuracy: 0.81
Best K: 11
Best Accuracy: 0.83


# **KNN MODELING**

In [ ]:
num_Neighbors=11
KNN=KNeighborsClassifier(n_neighbors=num_Neighbors)
KNN.fit(x_train,y_train)
KNN.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
test_predictions=KNN.predict(x_test)
print("\n")
print("Confusion matrix:\n",confusion_matrix(y_test,test_predictions))
print("\n")
print("Accuracy:\n",KNN.score(x_test, y_test))
print("\n")
print("Classification report on test :\n KNN, K=",
      num_Neighbors,"\n",
      classification_report(y_test, test_predictions))



Confusion matrix:
 [[1527   68]
 [ 266  139]]


Accuracy:
 0.833


Classification report on test :
 KNN, K= 11 
               precision    recall  f1-score   support

           0       0.85      0.96      0.90      1595
           1       0.67      0.34      0.45       405

    accuracy                           0.83      2000
   macro avg       0.76      0.65      0.68      2000
weighted avg       0.82      0.83      0.81      2000



In [ ]:
train_predictions=KNN.predict(x_train)
print("\n")
print("Confusion matrix:\n",confusion_matrix(y_train,train_predictions))
print("\n")
print("Accuracy:\n",KNN.score(x_train, y_train))
print("\n")
print("Classification report on train :\n KNN, K=",
      num_Neighbors,"\n",
      classification_report(y_train, train_predictions))



Confusion matrix:
 [[6193  175]
 [1081  551]]


Accuracy:
 0.843


Classification report on train :
 KNN, K= 11 
               precision    recall  f1-score   support

           0       0.85      0.97      0.91      6368
           1       0.76      0.34      0.47      1632

    accuracy                           0.84      8000
   macro avg       0.81      0.66      0.69      8000
weighted avg       0.83      0.84      0.82      8000



# **Decision Tree Model**

In [ ]:

dt_classifier = DecisionTreeClassifier(random_state=0)
dt_classifier.fit(x_train, y_train)
dt_predictions = dt_classifier.predict(x_test)
print("\nDecision Tree Classifier:")
print("Confusion matrix:\n", confusion_matrix(y_test, dt_predictions))
print("\nAccuracy:\n", dt_classifier.score(x_test, y_test))
print("\nClassification report on test:\n", classification_report(y_test, dt_predictions))





Decision Tree Classifier:
Confusion matrix:
 [[1377  218]
 [ 170  235]]

Accuracy:
 0.806

Classification report on test:
               precision    recall  f1-score   support

           0       0.89      0.86      0.88      1595
           1       0.52      0.58      0.55       405

    accuracy                           0.81      2000
   macro avg       0.70      0.72      0.71      2000
weighted avg       0.81      0.81      0.81      2000



In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=0)
dt_classifier.fit(x_test, y_test)
dt_predictions = dt_classifier.predict(x_train)
print("\nDecision Tree Classifier:")
print("Confusion matrix:\n", confusion_matrix(y_train, dt_predictions))
print("\nAccuracy:\n", dt_classifier.score(x_train, y_train))
print("\nClassification report on train:\n", classification_report(y_train, dt_predictions))


Decision Tree Classifier:
Confusion matrix:
 [[5513  855]
 [ 844  788]]

Accuracy:
 0.787625

Classification report on train:
               precision    recall  f1-score   support

           0       0.87      0.87      0.87      6368
           1       0.48      0.48      0.48      1632

    accuracy                           0.79      8000
   macro avg       0.67      0.67      0.67      8000
weighted avg       0.79      0.79      0.79      8000



# **Logitstic regression model**

In [ ]:
logreg_model = LogisticRegression(random_state=0)
logreg_model.fit(x_train, y_train)
logreg_predictions = logreg_model.predict(x_test)
print("\nLogistic Regression Classifier:")
print("Confusion matrix:\n", confusion_matrix(y_test, logreg_predictions))
print("\nAccuracy:\n", logreg_model.score(x_test, y_test))
print("\nClassification report on test:\n", classification_report(y_test, logreg_predictions))




Logistic Regression Classifier:
Confusion matrix:
 [[1538   57]
 [ 317   88]]

Accuracy:
 0.813

Classification report on test:
               precision    recall  f1-score   support

           0       0.83      0.96      0.89      1595
           1       0.61      0.22      0.32       405

    accuracy                           0.81      2000
   macro avg       0.72      0.59      0.61      2000
weighted avg       0.78      0.81      0.78      2000



In [ ]:
logreg_model = LogisticRegression(random_state=0)
logreg_model.fit(x_test, y_test)
logreg_predictions = logreg_model.predict(x_train)
print("\nLogistic Regression Classifier:")
print("Confusion matrix:\n", confusion_matrix(y_train, logreg_predictions))
print("\nAccuracy:\n", logreg_model.score(x_train, y_train))
print("\nClassification report on train:\n", classification_report(y_train, logreg_predictions))


Logistic Regression Classifier:
Confusion matrix:
 [[6229  139]
 [1361  271]]

Accuracy:
 0.8125

Classification report on train:
               precision    recall  f1-score   support

           0       0.82      0.98      0.89      6368
           1       0.66      0.17      0.27      1632

    accuracy                           0.81      8000
   macro avg       0.74      0.57      0.58      8000
weighted avg       0.79      0.81      0.76      8000



# **SVM MODEL**

In [ ]:
svm_classifier = SVC(kernel='linear', random_state=0)
svm_classifier.fit(x_train, y_train)
svm_predictions = svm_classifier.predict(x_test)
print("\nSVM Classifier:")
print("Confusion matrix:\n", confusion_matrix(y_test, svm_predictions))
print("\nAccuracy:\n", svm_classifier.score(x_test, y_test))
print("\nClassification report on test :\n", classification_report(y_test, svm_predictions))



SVM Classifier:
Confusion matrix:
 [[1595    0]
 [ 405    0]]

Accuracy:
 0.7975

Classification report on test :
               precision    recall  f1-score   support

           0       0.80      1.00      0.89      1595
           1       0.00      0.00      0.00       405

    accuracy                           0.80      2000
   macro avg       0.40      0.50      0.44      2000
weighted avg       0.64      0.80      0.71      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# אשית כל על מנת לבדוק אם המודל נמצא באוברפיטינג או באנדרפיטינג נבדוק את הדיוק
#גם על הטסט וגם על הטריין ואז נראה את ההפרשים בתוצאת הדיוק ,
# GOODFFITאם השנים דומים מאוד יש לנו
#אם הטריין גדול מידי מהטסט מודבר בOVERFITIING
#אם הטסט גדול מהטריין מדובר בUNDERFITIING
#לפי הסבר זה אפשר לראות שבמודל
#KNN-GOODFIT
#TREE-OVERFITTING
#LINIAR-GOODFIT
#SVM-GOODFIT

In [ ]:
#לפי התוצאות שקיבלתי הייתי משתמש במודל הKNN
#הוא הביא לי את הדיוק המירבי ונמצא בהתאמה טובה
#אני התסכלתי על מדד הRECALL
#בסופו של דבר חשוב ליח לחזות את הדבר הספציפי שאני רוצה ולכן חשוב לי שאחוז הRECALL יהיה גבוהה


In [ ]:
#ניסיתי לחזות בסופו של דבר האם לקוחות שמצטרפים אליי לבנק בסוף יעזבו אותי גם
#החיזוי הצליח לי ברמה יחסית וההצלחתי לחזות אותו ברמת ה34 אחוז
#שימוש בחיזוי זה יכול להיות מאוד ליעיל לבנקים בעת חשיבה על איך משמרים לקוחות ואיזה פעולות אפשר לדעת לפני על לקוח על מנת להשאיר אותו בתור לקוח הבנק
